In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
from ccfj import GetStationPairs
from geopy.distance import great_circle
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
#import shapefile
import geopandas as gp
import yaml
import math
#from pykml import parser


In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [ ]:
flag_project = 1 # 0--regular; 1--repartrition

In [ ]:
if flag_project == 0:
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
flag_fund = 1

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary


In [ ]:
dir_model_fund = dir_project + info_basic['dir_model_fund']
dir_model = dir_project + info_basic['dir_model']
dir_image = dir_project + info_basic['dir_image']+'Vs_compare/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_inv_dispernet = dir_project + info_basic['dir_inv_dispernet']
dir_partition = dir_project + info_basic['dir_partition']
key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

In [ ]:
struc = {}
struc_fund = {}
flag = 0
for key_subwork in key_subworks:
    file_model = dir_model + 'model_'+key_subwork+'.txt'
    file_model_fund = dir_model_fund + 'model_'+key_subwork+'.txt'
    model = np.loadtxt(file_model)
    model_fund = np.loadtxt(file_model_fund)
    struc[key_subwork] = {}
    struc[key_subwork]['layer'] = model[:, 0]
    struc[key_subwork]['z'] = model[:, 1]
    struc[key_subwork]['rho'] = model[:, 2]
    struc[key_subwork]['vs'] = model[:, 3]
    struc[key_subwork]['vp'] = model[:, 4]
    struc[key_subwork]['std'] = model[:, 5]
    struc_fund[key_subwork] = {}
    struc_fund[key_subwork]['layer'] = model_fund[:, 0]
    struc_fund[key_subwork]['z'] = model_fund[:, 1]
    struc_fund[key_subwork]['rho'] = model_fund[:, 2]
    struc_fund[key_subwork]['vs'] = model_fund[:, 3]
    struc_fund[key_subwork]['vp'] = model_fund[:, 4]
    struc_fund[key_subwork]['std'] = model_fund[:, 5]
    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 

In [ ]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
num_stations = []
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    num_stations.append(len(stations_this))
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

### Plot horizontal of certain depth

In [ ]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [ ]:
#%%capture
M = 100
dz = 5
N = 60


d = -27.3

for i in range(N):

    vs_all = []
    vs_all_fund = []
    for key_subwork in key_subworks:
        vs_all.append(struc[key_subwork]['vs'][i])
        vs_all_fund.append(struc_fund[key_subwork]['vs'][i])


    if i*dz <= 30:
        c_min = 0.3
        c_max = 0.6
    elif i*dz <= 60:
        c_min = 0.5
        c_max = 0.8
    elif i*dz < 120:
        c_min = 0.7
        c_max = 1.1
    else:
        c_min = 0.8
        c_max = 1.3 

    fig = plt.figure(figsize=(20, 14))
    ax = fig.add_subplot(221)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    flag_add = 0.0005
    ax.set_xlim([min(lon_stations_all)-flag_add, max(lon_stations_all)+flag_add])
    ax.set_ylim([min(lat_stations_all)-flag_add, max(lat_stations_all)+flag_add])
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    im = ax.scatter(lon_centroid_partition,lat_centroid_partition,c=vs_all_fund,cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    ax = fig.add_subplot(222)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Overtones')
    flag_add = 0.0005
    ax.set_xlim([min(lon_stations_all)-flag_add, max(lon_stations_all)+flag_add])
    ax.set_ylim([min(lat_stations_all)-flag_add, max(lat_stations_all)+flag_add])
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    im = ax.scatter(lon_centroid_partition,lat_centroid_partition,c=vs_all,cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    ax = fig.add_subplot(223)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    flag_add = 0.0005
    ax.set_xlim([min(lon_stations_all)-flag_add, max(lon_stations_all)+flag_add])
    ax.set_ylim([min(lat_stations_all)-flag_add, max(lat_stations_all)+flag_add])
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    im = ax.scatter(lon_centroid_partition,lat_centroid_partition,c=vs_all_fund,vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    ax = fig.add_subplot(224)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Overtones')
    flag_add = 0.0005
    ax.set_xlim([min(lon_stations_all)-flag_add, max(lon_stations_all)+flag_add])
    ax.set_ylim([min(lat_stations_all)-flag_add, max(lat_stations_all)+flag_add])
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    im = ax.scatter(lon_centroid_partition,lat_centroid_partition,c=vs_all,vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    filename = dir_image + 'Vs_hori_profile_'+str((i+1)*dz)+'m_compare.png'
    plt.savefig(filename,dpi=100)
    plt.close()